<a href="https://colab.research.google.com/github/JeyTee1988/LLM-Schulung/blob/main/llm_schulung.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Zunächst installieren wir die benötigte Library Huggingface Transformer

Flo jetzt aber hier

In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 33.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 61.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.5 MB/s eta 0:00:00


Dann downloaden wir das vortrainierte distilgpt2 Modell

In [ ]:
from transformers import pipeline

pipe = pipeline("text-generation", model="distilgpt2")

# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("distilgpt2")
model = AutoModelForCausalLM.from_pretrained("distilgpt2")

Wir können das Modell nutzen, um Texte zu vervollständigen.

In [ ]:
from transformers import set_seed
set_seed(42)
pipe("Hello, I’m a language model", max_length=20, num_return_sequences=5)



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Hello, I’m a language model but what I do in that role is to make everything'},
 {'generated_text': 'Hello, I’m a language model.” You’ll know the real name'},
 {'generated_text': 'Hello, I’m a language model’s\xadself, and I’m'},
 {'generated_text': 'Hello, I’m a language model, and I really wanted to make a nice API,'},
 {'generated_text': 'Hello, I’m a language model. I’think in any language. I�'}]

Ein Tokenizer in einem Language Learning Model (LLM) dient dazu, Text in kleinere Einheiten, sogenannte Tokens, zu zerlegen. Diese Tokens können einzelne Wörter, Phrasen oder sogar einzelne Buchstaben sein.

Der Tokenizer ist wichtig, weil ein LLM wie ein neuronales Netzwerk arbeitet, das auf Zahlen, nicht auf Wörtern, basiert. Es kann nicht direkt Text verstehen oder interpretieren. Stattdessen muss es den Text in eine numerische Form umwandeln, die es verarbeiten kann.

Dieser Prozess des Zerlegens und Umwandeln wird durch den Tokenizer durchgeführt. Er nimmt den ursprünglichen Text, teilt ihn in Tokens und wandelt diese dann in Zahlen um, die das Modell verarbeiten kann.

https://platform.openai.com/tokenizer


In [ ]:
text = "Hello, I’m a language model"
encoded_input = tokenizer(text, return_tensors='pt')
encoded_input.input_ids

tensor([[15496,    11,   314,   447,   247,    76,   257,  3303,  2746]])